In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/wuxinchao/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# %pip install openpyxl

In [4]:
# load excel file data
df = pd.read_excel("~/data/project/pMHC-TCR/230213_info.xlsx")

In [16]:
df["nSeqHLA"].value_counts()

ATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGCTCCCACTCCATGAGGTATTTCTACACCTCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAGGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATGTGAAGGCCCAGTCACAGACTGACCGAGTGGACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTTCTCACACCATCCAGATAATGTATGGCTGCGACGTGGGGCCGGACGGGCGCTTACTCCGCGGGTACCGGCAGGACGCCTACGACGGCAAGGATTACATCGCCCTGAACGAGGACCTGCGCTCTTGGACCGCGGCGGACATGGCAGCTCAGATCACCAAGCGCAAGTGGGAGGCGGCCCATGCGGCGGAGCAGCAGAGAGCCTACCTGGAGGGCCGGTGCGTGGAGTGGCTCCGCAGATACCTGGAGAACGGGAAGGAGACGCTGCAGCGCACGGACCCCCCCAAGACACATATGACCCACCACCCCATCTCTGACCATGAGGCCACCCTGAGGTGCTGGGCCCTGGGCTTCTACCCTGCGGAGATCACACTGACCTGGCAGCGGGATGGGGAGGACCAGACCCAGGACACGGAGCTCGTGGAGACCAGGCCTGCAGGGGATGGAACCTTCCAGAAGTGGGCGGCTGTGGTGGTGCCTTCTGGAGAGGAGCAGAGATACACCTGCCATGTGCAGCATGAGGGTCTGCCCAAGCCCCTCACCCTGAGATGGGAGCTGTCTTCCCAGCCCACCATCCCCATCGTGGGCATCATTGCTGGCCTGGTTCTCCTTGGAGCTGTGATCACTGGAGCTGTGGTCGCTGCCGTGATGTGGAGGA

In [50]:
# reindex the df by cellname and chain
# df.sort_values(by=['cellname', "chain"])
df = df.set_index(['cellname', "chain"])
# extract the NeoAA, HLA, and aaSeqCDR columns
df = df[["NeoAA", "HLA", "aaSeqCDR1", "aaSeqCDR2", "aaSeqCDR3"]]
# df["aaSeqCDR1"].value_counts()
df["aaSeqCDR"] = df[df.columns[2:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [125]:
idx = pd.IndexSlice

df_a = df.loc[idx[:,"TRA"],]
df_a["AseqCDR"] = df_a["aaSeqCDR"]
df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_a.index = df_a.index.droplevel(1)
# print(df_a)
df_b = df.loc[idx[:,"TRB"],]
df_b["BseqCDR"] = df_b["aaSeqCDR"]
df_b.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
# drop the chain index
df_b.index = df_b.index.droplevel(1)
# print(df_b)

# merge the TRA and TRB dataframes by cellname, HLAs, and NeoAA
df_ab = pd.merge(df_a, df_b, on=["cellname", "HLA", "NeoAA"])
df_ab
# df_ab = pd.merge(df_a, df_b, on="cellname")
# df_ab

/tmp/ipykernel_4163421/3989028287.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["AseqCDR"] = df_a["aaSeqCDR"]
/tmp/ipykernel_4163421/3989028287.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a.drop(columns=["aaSeqCDR","aaSeqCDR1","aaSeqCDR2","aaSeqCDR3"], inplace=True)
/tmp/ipykernel_4163421/3989028287.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,NeoAA,HLA,AseqCDR,BseqCDR
cellname,,,,
V350110758_L02_532,VVVGAGDVGK,HLA-A*11:01,"NSMFDY,ISSIKDK,CAASAGTYKYIF","SGHTS,YDEGEE,CASSLVSSYEQYF"
V350110758_L02_531,VVVGAGDVGK,HLA-A*11:01,"DSAIYN,IQSSQRE,CAVRGNNNARLMF","GTSNPN,SVGIG,CAWSPGRSPLHF"
V350110758_L02_523,VVVGAGDVGK,HLA-A*11:01,"SSNFYA,MTLNGDE,CAFYGSARQLTF","PRHDT,FYEKMQ,CASSPPTTAANYGYTF"
V350110758_L02_522,VVVGAGDVGK,HLA-A*11:01,"NSMFDY,ISSIKDK,CAASGTGTASKLTF","SGHNS,FNNNVP,CASETSNNEQFF"
V350110758_L02_520,TTAPPLSGK,HLA-A*11:01,"SGHDT,YYEEEE,CASSLADQETQYF","SGHDT,YYEEEE,CASSLADQETQYF"
...,...,...,...,...
V350085868_L01_506,VLLSHLSYL,HLA-A*02:01,"ATGYPS,ATKADDK,CALTVSYGGSQGNLIF","KGHSH,LQKENI,CASSPFSIGQGLTNNEKLFF"
V350085868_L01_505,VLLSHLSYL,HLA-A*02:01,"NIATNDY,GYKTK,CLVGTNSNSGYALNF","LGHNT,FRNRAP,CASGSPDRFEQYF"
V350085868_L01_504,VLLSHLSYL,HLA-A*02:01,"DFQATT,SNEGSKA,CSVRNGYEQYF","DFQATT,SNEGSKA,CSVRNGYEQYF"


In [85]:
# not used
# (df['aaSeqCDR'].value_counts() > 1).sum()
count = 0
for cell in df.index:
    rep_dict = {}
    if df.loc[(cell), "aaSeqCDR"] in rep_dict:
        count += 1
    else:
        rep_dict[df.loc[(cell), "aaSeqCDR"]] = 1
print(rep_dict)
print(count)

{'SGHTA,FQGNSA,CASSLGVYEQYF': 1}
0


In [47]:
# not used
# sorting data: combine the sequence from the same cell (has the same cellname), 
# add the prefix of the column chain to the columns CDR1, CDR2, CDR3
data = df.groupby("cellname").agg(
    {
        "NeoGeneID": "first",
        "NeoAA" : "first",
        "HLA": "first",
        "chain": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR1": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR2": lambda x: x.iloc[0] + " " + x.iloc[1],
        "aaSeqCDR3": lambda x: x.iloc[0] + " " + x.iloc[1],
    }
)
# split each aaSeqCDR1 into 2 columns, one for A chain and one for B chain
data[['aaSeqCDR1_B', 'aaSeqCDR1_A']] = data.aaSeqCDR1.str.split(" ",expand=True,)
data[['aaSeqCDR2_B', 'aaSeqCDR2_A']] = data.aaSeqCDR2.str.split(" ",expand=True,)
data[['aaSeqCDR3_B', 'aaSeqCDR3_A']] = data.aaSeqCDR3.str.split(" ",expand=True,)
data.drop(columns=['aaSeqCDR1', 'aaSeqCDR2', 'aaSeqCDR3'], inplace=True)

In [97]:
# not used
df.groupby(['aaSeqCDR', 'chain', 'cellname']).agg(
    {
        # find the TRA and TRB chains of the same cell has the same aaSeqCDR
        'aaSeqCDR': 'count',
    }
)

aaSeqCDR
aaSeqCDR                         chain cellname                    
ATGYPS,ATKADDK,CALRGGATNKLIF     TRA   V350087414_L01_559         1
ATGYPS,ATKADDK,CALSANAGGTSYGKLTF TRA   V350087414_L01_543         1
                                       V350087414_L01_544         1
ATGYPS,ATKADDK,CALSDLASKIIF      TRA   V350085868_L02_524         1
ATGYPS,ATKADDK,CALSDRG_GSEKLVF   TRA   V350087414_L03_540         1
...                                                             ...
YSGSPE,HISR,CALSGLTGGGNKLTF      TRA   V350085868_L02_534         1
YSGSPE,HISR,CALSGRIV_GSNYKLTF    TRA   V350110749_L02_504         1
YSGSPE,HISR,CALTVSNYGGSQGNLIF    TRA   V350087584_L03_507         1
YSGSPE,HISR,CAPRSGGSNYKLTF       TRA   V350087414_L01_524         1
YSGSPE,HISR,CA_GF                TRA   V350087393_L04_557         1

[2130 rows x 1 columns]

In [25]:
# torch dataset
class TCRDataset(Dataset):
    '''
    Use each two rows of data as a sample (one for alpha chain, one for beta chain), each sample has the common cellname
    Different from the chain, the HLA class is the same for each sample
    The aaSeqCDR1, aaSeqCDR2, aaSeqCDR3 are the CDR1, CDR2, CDR3 of the alpha chain and beta chain, respectively.
    '''
    def __init__(self):
        self.root = "~/data/project/pMHC-TCR/230213_info.xlsx"
        # self.data = data
        # # each sample has two rows of data, one for alpha chain, one for beta chain
        # self.len = len(self.data) / 2
        # # aggregate the cellname
        # self.cellname = self.data["cellname"]
        # # aggregate the NeoAA
        # self.neoAA = self.data["NeoAA"]
        # # aggregate the HLA class
        # self.HLA = self.data["HLA"]
        # # aggregate the CDR1, CDR2, CDR3 of the alpha chain and beta chain, respectively
        # self.aCDR1 = self.data["aaSeqCDR1"] if self.data["chain"] == "A" else None
        # self.aCDR2 = self.data["aaSeqCDR2"] if self.data["chain"] == "A" else None
        # self.aCDR3 = self.data["aaSeqCDR3"] if self.data["chain"] == "A" else None
        # self.bCDR1 = self.data["aaSeqCDR1"] if self.data["chain"] == "B" else None
        # self.bCDR2 = self.data["aaSeqCDR2"] if self.data["chain"] == "B" else None
        # self.bCDR3 = self.data["aaSeqCDR3"] if self.data["chain"] == "B" else None